# LLM-Powered ISO 8583 Features (Ollama - Local LLM)

This notebook demonstrates the AI-powered features using **Ollama** for local, private LLM inference.

No API keys needed - runs completely offline!

## Prerequisites

1. Install Ollama: https://ollama.com
2. Start the service: `ollama serve`
3. Pull a model: `ollama pull llama3.1` or `ollama pull mistral`
4. Install the Python package: `pip install iso8583sim[ollama]`

In [1]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.types import ISO8583Message
from iso8583sim.demo import generate_auth_request, generate_emv_auth, pretty_print

## Check Available Providers

First, let's see which LLM providers are available:

In [2]:
from iso8583sim.llm import list_available_providers, list_installed_providers

print("Installed providers (packages installed):")
print(f"  {list_installed_providers() or 'None'}")

print("\nAvailable providers (installed + configured):")
print(f"  {list_available_providers() or 'None - set API key env var'}")

Installed providers (packages installed):


  ['anthropic', 'openai', 'google', 'ollama']

Available providers (installed + configured):
  ['ollama']


## MessageExplainer

The `MessageExplainer` takes an ISO 8583 message and provides a human-readable explanation.

In [3]:
from iso8583sim.llm import MessageExplainer
from iso8583sim.llm.providers.ollama import OllamaProvider

# Create an explainer with specific Ollama model
try:
    provider = OllamaProvider(model="qwen3-coder:30b")
    explainer = MessageExplainer(provider=provider)
    print(f"Using provider: {explainer.provider.name} / {explainer.provider.model}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    explainer = None

Using provider: Ollama / qwen3-coder:30b


### Explain a Message Object

In [4]:
# Create a sample message
message = generate_auth_request(
    pan="4111111111111111",
    amount=10000,  # $100.00
    terminal_id="TERM0001",
    merchant_id="GASSTATION12345",
)

print("Message to explain:")
pretty_print(message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation:")
    print("=" * 60)
    explanation = explainer.explain(message)
    print(explanation)

Message to explain:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (9 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 000000
  F004 [NUMERIC] Amount, Transaction                 = 000000010000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2612
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = TERM0001
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = GASSTATION12345
  F049 [NUMERIC] Currency Code, Transaction          = 840

LLM Explanation:


### Transaction Analysis

**1. Transaction Type**
- **MTI 0100**: Request message from an **Acquirer** (digit 1 = 0), for a **Financial** transaction (digit 2 = 1), **Request** (digit 3 = 0), from **Acquirer** (digit 4 = 0)
- **Processing Code 000000**: Indicates a **Purchase** transaction (first two digits 00 = Purchase, next two 00 = not cashback, last two 00 = not refund)

**2. Key Details**
- **Card Number**: 411111******1111 (VISA card, PAN starts with 4)
- **Transaction Amount**: $100.00 (4 digits = 10000, so 100.00 in minor units)
- **STAN (System Trace Audit Number)**: 123456
- **Expiration Date**: 26/12 (December 2026)
- **POS Entry Mode**: 051 (Chip card transaction)
- **Terminal ID**: TERM0001
- **Merchant ID**: GASSTATION12345
- **Currency Code**: 840 (USD)

**3. Notable Aspects / Potential Issues**
- **Card Type**: VISA (starts with 4)
- **Entry Mode**: Chip card (051) – secure transaction
- **No Track 2 or PIN**: No F35 (Track 2) or F55 (EMV data) fields present, which ma

### Explain a Raw Message String

In [5]:
builder = ISO8583Builder()
raw_message = builder.build(message)

print(f"Raw message: {raw_message[:80]}...")

if explainer:
    print("\nExplanation:")
    explanation = explainer.explain(raw_message)
    print(explanation)

Raw message: 01007024040000C080001641111111111111110000000000000100001234562612051TERM0001GAS...

Explanation:


### Transaction Analysis

**1. Type of Transaction:**
- **MTI 0100** indicates:
  - Version: 0 (1987 standard)
  - Class: 1 (Authorization request)
  - Function: 0 (Request)
  - Origin: 0 (Acquirer)
- **Processing Code (F003): 000000** indicates a **purchase transaction** (standard retail purchase)
- This is an **authorization request** for a card payment

**2. Key Details:**
- **Amount:** $100.00 (F004: 000000010000 = 10000 cents = $100.00)
- **Card Number:** 411111******1111 (VISA card)
- **Card Type:** VISA (starts with 4)
- **Terminal ID:** TERM0001
- **Merchant ID:** GASSTATION12345
- **Merchant Name:** GAS STATION (location)
- **Expiration Date:** 12/26 (F014: 2612 = Dec 2026)
- **Entry Mode:** Chip (F022: 051 = Chip card)
- **STAN (System Trace Audit Number):** 123456

**3. Notable Aspects:**
- **Card Type:** VISA (411111...), which is a test card number (not a real card)
- **Entry Mode:** Chip card transaction (secure)
- **Amount:** $100.00 (a relatively large purchase)
- **Mer

### Explain EMV/Chip Card Message

In [6]:
emv_message = generate_emv_auth(
    pan="4111111111111111",
    amount=25000,  # $250.00
    cryptogram="1234567890ABCDEF",
)

print("EMV Message:")
pretty_print(emv_message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation of EMV Transaction:")
    print("=" * 60)
    explanation = explainer.explain(emv_message, verbose=True)
    print(explanation)

EMV Message:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (12 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 000000
  F004 [NUMERIC] Amount, Transaction                 = 000000025000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2612
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F023 [NUMERIC] Card Sequence Number                = 001
  F035 [LLVAR] Track 2 Data                        = 4111111111111111=26125010000000000000
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = TERM0001
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = MERCHANT123456 
  F049 [NUMERIC] Currency Code, Transaction          = 840
  F055 [LLLVAR] IC

### 1. **Transaction Type (Based on MTI and Processing Code)**

- **MTI: 0100**
  - **Digit 1 (0)**: ISO8583:1987 version
  - **Digit 2 (1)**: Authorization message (class 1)
  - **Digit 3 (0)**: Request (function 0)
  - **Digit 4 (0)**: Acquirer origin (origin 0)
  - **Meaning**: This is an **authorization request** from an acquirer to an issuer (e.g., a point-of-sale terminal sending a purchase request).

- **Processing Code (F003): 000000**
  - **First 2 digits (00)**: Purchase (standard retail transaction)
  - **Next 2 digits (00)**: Not specified, likely standard card-present transaction
  - **Last 2 digits (00)**: Not specified, likely standard card-present transaction
  - **Meaning**: This is a **standard purchase transaction**.

---

### 2. **Key Details**

- **Card Number (F002): 411111******1111**
  - **Card Type**: VISA (starts with 4)
  - **Masked PAN**: First 6 and last 4 digits shown; middle digits are masked

- **Transaction Amount (F004): 000000025000**
  - **Formatted 

### Explain Specific Fields

In [7]:
if explainer:
    # Explain response codes
    print("Explaining Response Code 51:")
    print(explainer.explain_response_code("51"))

    print("\n" + "=" * 40)

    # Explain a specific field
    print("\nExplaining Field 22 (POS Entry Mode):")
    print(explainer.explain_field(22, "051"))

Explaining Response Code 51:


## ISO 8583 Response Code "51" - Insufficient Funds

### 1. What this code means

**Response Code 51** indicates that the transaction has been declined because the cardholder's account does not have sufficient funds to cover the requested amount. This is one of the most common decline reasons in card processing and represents a fundamental financial constraint at the account level.

### 2. Common scenarios when this response is returned

**Typical situations include:**

- **Overdraft scenarios**: Cardholder attempts to spend more than their available balance
- **Credit card transactions**: Purchase exceeds available credit limit
- **Debit card transactions**: ATM withdrawal or purchase exceeds account balance
- **Pre-authorization holds**: Amount reserved exceeds available funds
- **Multiple concurrent transactions**: Previous transactions have already consumed available balance
- **Account restrictions**: Temporary holds or account freezes due to suspicious activity
- **Currency conve

## Field 22: Point of Service Entry Mode

### 1. What this field represents
Field 22 (POS Entry Mode) identifies how the card data was entered at the point of sale terminal. It indicates the method and capabilities of the card reader/device used for transaction processing.

### 2. How to interpret the specific value "051"
The value **051** is a 3-digit code broken down as follows:
- **First digit (0)**: Card data input method - "0" = Magnetic stripe
- **Second digit (5)**: Card data authentication - "5" = Chip card with PIN
- **Third digit (1)**: Card data capture - "1" = Cardholder present

### 3. Relevant context and meaning
This is a **POS Entry Mode code** that follows the ISO 8583 standard for card transaction entry methods. Each digit represents a specific aspect of the transaction:

**First digit (Card data input):**
- 0 = Magnetic stripe
- 1 = Chip card (EMV)
- 2 = Manual entry
- 3 = Contactless
- 4 = ICC (Integrated Circuit Card)
- 5 = Chip card with PIN
- 6 = Chip card withou

## MessageGenerator

The `MessageGenerator` creates ISO 8583 messages from natural language descriptions.

In [8]:
from iso8583sim.llm import MessageGenerator
from iso8583sim.llm.providers.ollama import OllamaProvider

try:
    provider = OllamaProvider(model="qwen3-coder:30b")
    generator = MessageGenerator(provider=provider)
    print(f"Using provider: {generator.provider.name} / {generator.provider.model}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    generator = None

Using provider: Ollama / qwen3-coder:30b


### Generate from Natural Language

In [9]:
if generator:
    # Generate a simple purchase
    description = "$50 VISA purchase at a coffee shop"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: $50 VISA purchase at a coffee shop

Generating message...


Generation failed: Failed to parse LLM response as JSON: Unterminated string starting at: line 20 column 11 (char 414)
Response: {
  "mti": "0100",
  "fields": {
    "2": "4111111111111111",
    "3": "000000",
    "4": "000000050000",
    "11": "000001",
    "12": "123456",
    "13": "0123",
    "14": "2512",
    "22": "050",
    "32": "123456",
    "35": "4111111111111111D251212345678901234567890",
    "38": "123456",
    "39": "00",
    "41": "TERMINAL01",
    "42": "MERCHANT123456",
    "43": "COFFEE SHOP",
    "49": "840",
    "55": "30000000000000000000000000000000


In [10]:
if generator:
    # Generate a refund
    description = "Refund $25 to Mastercard ending in 4444 at ACME Store"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Refund Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: Refund $25 to Mastercard ending in 4444 at ACME Store

Generating message...


Generation failed: Failed to parse LLM response as JSON: Unterminated string starting at: line 12 column 11 (char 208)
Response: {
  "mti": "0100",
  "fields": {
    "2": "4111111111111111",
    "3": "200000",
    "4": "000000025000",
    "11": "000001",
    "12": "123456",
    "13": "0101",
    "14": "2512",
    "22": "05",
    "35": "4111111111111111D25120000000000000000000000000000000


In [11]:
if generator:
    # Generate a balance inquiry
    description = "Balance inquiry for VISA card at ATM"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Balance Inquiry:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: Balance inquiry for VISA card at ATM

Generating message...


Generation failed: Failed to parse LLM response as JSON: Unterminated string starting at: line 14 column 11 (char 258)
Response: {
  "mti": "0100",
  "fields": {
    "2": "4111111111111111",
    "3": "000000",
    "4": "000000000000",
    "11": "000001",
    "12": "123456",
    "13": "0101",
    "14": "2512",
    "22": "05",
    "32": "00000000000",
    "33": "00000000000",
    "35": "4111111111111111D25120000000000000000000000000000000


### Suggest Missing Fields

In [12]:
if generator:
    # Create a partial message
    partial_message = ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: "4111111111111111",
            4: "000000010000",
        },
    )

    print("Partial Message:")
    print(f"  MTI: {partial_message.mti}")
    print(f"  Fields: {list(partial_message.fields.keys())}")

    print("\nSuggesting missing fields...")
    suggestions = generator.suggest_fields(partial_message)

    if suggestions:
        print("\nSuggested Fields:")
        for field_num, value in sorted(suggestions.items()):
            print(f"  F{field_num:03d}: {value}")
    else:
        print("No suggestions available")

Partial Message:
  MTI: 0100
  Fields: [0, 2, 4]

Suggesting missing fields...


No suggestions available


## Using Specific Providers

You can specify which LLM provider to use:

In [13]:
# Use a specific provider by name
try:
    from iso8583sim.llm import get_provider

    # Try different providers
    for provider_name in ["anthropic", "openai", "google", "ollama"]:
        try:
            provider = get_provider(provider_name)
            print(f"✓ {provider_name}: {provider.model}")
        except Exception as e:
            print(f"✗ {provider_name}: {e}")
except Exception as e:
    print(f"Error: {e}")

✗ anthropic: Anthropic provider requires 'anthropic' package. Install with: pip install iso8583sim[anthropic]
✗ openai: OpenAI API key not found. Set OPENAI_API_KEY environment variable or pass api_key parameter.
✗ google: Google provider requires 'google-generativeai' package. Install with: pip install iso8583sim[google]
✓ ollama: llama3.2


In [14]:
# Use provider with custom settings
try:
    from iso8583sim.llm.providers.anthropic import AnthropicProvider

    # Use a specific model
    provider = AnthropicProvider(model="claude-3-haiku-20240307")
    explainer = MessageExplainer(provider=provider)

    print(f"Using: {explainer.provider.name} / {explainer.provider.model}")
except Exception as e:
    print(f"Could not create custom provider: {e}")

Could not create custom provider: Anthropic provider requires 'anthropic' package. Install with: pip install iso8583sim[anthropic]


## Error Handling

The LLM module provides clear error messages:

In [15]:
from iso8583sim.llm import LLMError, ProviderConfigError, ProviderNotAvailableError

# Example error handling
try:
    from iso8583sim.llm import get_provider

    provider = get_provider("invalid_provider")
except ProviderConfigError as e:
    print(f"Config Error: {e}")
except ProviderNotAvailableError as e:
    print(f"Not Available: {e}")
except LLMError as e:
    print(f"LLM Error: {e}")

Config Error: Unknown provider: invalid_provider. Available: anthropic, openai, google, ollama


## Next Steps

- Try different LLM providers to see how they compare
- Use the generator to create test message datasets
- Combine with validation to ensure generated messages are correct
- Check out the API reference for more advanced usage